In [ ]:
!pip install sqlite3

In [2]:
def search_by_filename(conn, filename):
    cursor = conn.cursor()
    query = '''
    SELECT 
        f.file_id,
        f.name AS filename,
        GROUP_CONCAT(DISTINCT k.keyw_id) AS keyword_ids,
        GROUP_CONCAT(DISTINCT mv.value) AS metadata_values
    FROM 
        files f
    LEFT JOIN 
        keywords k ON f.file_id = k.file_id
    LEFT JOIN 
        metadata m ON f.file_id = m.file_id
    LEFT JOIN 
        metadata_values mv ON m.value_id = mv.id
    WHERE 
        f.name = ?
    GROUP BY
        f.file_id, f.name
    '''
    cursor.execute(query, (filename,))
    return cursor.fetchall()

def process_search_results(filename, conn):
    results = search_by_filename(conn, filename)

    if results:
        print(f"Results for '{filename}':")
        for row in results:
            print(f"File ID: {row[0]}, Filename: {row[1]}")
            print("Keyword IDs:", row[2])
            print("Metadata Values:", row[3])
            
            metadata_values = row[3].split(',')
            
            # Extract Character keywords and remove prefixes
            character_keywords = [value.strip().split('|')[1] for value in metadata_values if value.strip().startswith('Character|')]
            
            # Extract and separate Sounds keywords
            sounds_keywords = [value.strip().split('|')[1:] for value in metadata_values if value.strip().startswith('Sounds|')]
            sounds1_keywords = list(set(kw[0] for kw in sounds_keywords))
            sounds2_keywords = list(set(kw[1] for kw in sounds_keywords if len(kw) > 1))
            
            # Extract base instrument
            base_instrument = next((value.strip() for value in metadata_values if value.strip() in ['Wavetable', 'Analog', 'Operator', 'Sampler']), None)
            
            # Determine if it's an instrument rack
            is_instrument_rack = 'Instrument Rack' in metadata_values
            
            print("\nCharacter Keywords:", character_keywords)
            print("Sounds1 Keywords:", sounds1_keywords)
            print("Sounds2 Keywords:", sounds2_keywords)
            print("Base Instrument:", base_instrument)
            print("Is Instrument Rack:", is_instrument_rack)
            print()
    else:
        print(f"No results found for '{filename}'")



In [3]:
import sqlite3

# Create a connection to the SQLite database
conn = sqlite3.connect('./Live-files-1218.db')

print("Connected to the database successfully.")


Connected to the database successfully.


In [5]:
filename = "Don't Laugh.adv"
process_search_results(filename, conn)

Results for 'Don't Laugh.adv':
File ID: 7196, Filename: Don't Laugh.adv
Keyword IDs: 8596,8620
Metadata Values: Created by: Felix Zoepf,device:ableton:instr:InstrumentMeld,Meld,Instrument,Character|Digital,Sounds|Ambience & FX|Sound FX

Character Keywords: ['Digital']
Sounds1 Keywords: ['Ambience & FX']
Sounds2 Keywords: ['Sound FX']
Base Instrument: None
Is Instrument Rack: False

